# Machine Learning para DOA - Dregon Dataset

O objetivo aqui é utilizar regrassão para determinar a direção de chegada. Para isso, já extraí os delays referentes a todas as combinações de microfones do Dregon Dataset - Clean Speech.

O algoritmo usado para extrair os delays foi o delayDatasetCreator.py. Pode ser que a informação de todos os delays seja redundante, uma vez que o algoritmo determinístico usa só os delays em relação ao microfone da origem do espaço vetorial.

In [1]:
import pandas as pd
import numpy as np
from math import sqrt, cos, sin, pi
from sklearn.linear_model import LinearRegression, LassoLars, RANSACRegressor
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse, r2_score as r2

#### Abrindo o CSV com Pandas e separando as features das classificações 

O dataset tem as seguintes colunas:

Nome do arquivo WAV, Features, Azimutal Real, Elevação Real

Eu vou nomear os vetores com "teste" já que eu vou treinar o regressor com dados fictícios. Os dados do dataset serão todos de teste.

In [9]:
dataset       = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregonDelaysDataset.csv")
xTest         = np.array(dataset[dataset.columns[1:-2]])
yTestAzimutal = np.array(dataset[dataset.columns[-2]])
yTestElevacao = np.array(dataset[dataset.columns[-1]])

In [3]:
print(xTest[0], yTestAzimutal[0], yTestElevacao[0])

[ 12.  12.  22.  18.  17.   7.   6.  -1.  10.   6.   4.  -5.  -6.  11.
   7.   5.  -5.  -6.  -5.  -6. -16. -17.  -1. -11. -12.  -9. -10.  -1.] 45 -30


#### Definindo algumas funções que vou precisar

In [4]:
def radParaGrau(angulo):
    return (angulo*180)/pi

def grauParaRad(angulo):
    return (angulo*pi)/180

def vetorUnitario(vetor):
    return vetor/np.linalg.norm(vetor)

def tempoParaAmostras(tempo, freqAmostragem):
    return tempo * freqAmostragem

#### Criando os dados fictícios de treino

Os dados de treino não serão dados reais. Vou usar a conta do produto interno para verificar qual seria o delay ideal entre cada microfone. Assim, posso criar dados fictícios que representam o caso ideal, e com eles, treinar os regressores.

http://www.labbookpages.co.uk/audio/beamforming/delayCalc.html

In [5]:
# COORDENADAS ORIGINAIS
coordenadasMics = np.array([
    [0.0420, 0.0615, -0.0410],
    [-0.0420, 0.0615, 0.0410],
    [-0.0615, 0.0420, -0.0410],
    [-0.0615, -0.0420, 0.0410],
    [-0.0420, -0.0615, -0.0410],
    [0.0420, -0.0615, 0.0410],
    [0.0615, -0.0420, -0.0410],
    [0.0615, 0.0420, 0.0410]
])

In [22]:
# CRIANDO OS ARRAYS DE XTRAIN E YTRAIN
xTrain         = []
yTrainAzimutal = []
yTrainElevacao = []

# VOU PRECISAR DA FREQUENCIA DE AMOSTRAGEM DO DATASET
freqAmostragem = 44100

# VOU CALCULAR OS DELAYS COMO SE O SOM TIVESSE VINDO DAS SEGUINTES COMBINACOES
# DE AZIMUTAIS E ELEVACOES

azimutaisDesejados = [0,10,20,30,40,50,60,70,80,90]
elevacoesDesejadas = [-90,-80,-70,-60,-50,-40,-30,-20,-10,0]
for azimutalAtual in azimutaisDesejados:
    for elevacaoAtual in elevacoesDesejadas:
        
        # COLOCANDO NOS ARRAYS DE YTRAIN
        yTrainAzimutal.append(azimutalAtual)
        yTrainElevacao.append(elevacaoAtual)
        
        # TENHO QUE PASSAR PRA RADIANOS PRA FAZER AS CONTAS
        azimutalAtualRad = grauParaRad(azimutalAtual)
        elevacaoAtualRad = grauParaRad(elevacaoAtual)

        # PARA CADA COMBINAÇÃO DE MICROFONES
        linhaAtualXTrain = []
        for micI in range(0, 8):
            for micJ in range(micI + 1, 8):

                # COORDENADAS DA DIFERENCA DOS MICS
                coordenadasDiferenca = coordenadasMics[micI] - coordenadasMics[micJ]

                # COORDENADAS DO VETOR WAVEFRONT (JA VOU DEIXAR ELE UNITARIO)
                w = vetorUnitario(np.array([
                    cos(azimutalAtualRad)*cos(elevacaoAtualRad),
                    sin(azimutalAtualRad)*cos(elevacaoAtualRad),
                    sin(elevacaoAtualRad)
                ]))

                # CALCULANDO O PRODUTO INTERNO E DIVIDINDO PELA VELOCIDADE DO SOM
                delayTemporal = coordenadasDiferenca[0] * w[0]
                delayTemporal += coordenadasDiferenca[1] * w[1]
                delayTemporal += coordenadasDiferenca[2] * w[2]
                delayTemporal /= 340

                # VOU COLOCANDO OS RESULTADOS NA LINHA ATUAL DE XTRAIN
                linhaAtualXTrain.append(tempoParaAmostras(delayTemporal, freqAmostragem))
        
        # AGORA, JA POSSO COLOCAR A LINHA INTEIRA EM XTRAIN
        xTrain.append(linhaAtualXTrain)
        
# PASSANDO TUDO PRA NUMPY
xTrain = np.array(xTrain).astype(int)
yTrainAzimutal = np.array(yTrainAzimutal)
yTrainElevacao = np.array(yTrainElevacao)

#### Treinando o regressor e prezidendo dados

In [23]:
# PRIMEIRO COM O AZIMUTAL
regressorAzimutal = LinearRegression().fit(xTrain, yTrainAzimutal)
yPredAzimutal = regressorAzimutal.predict(xTest)

for i, predicao in enumerate(yPredAzimutal):
    print(yTestAzimutal[i], predicao)

45 55.63241747408111
90 88.49189769645386
60 56.58551317587524
75 82.86782914247752
60 68.36803847514072
45 39.07923320253238
60 72.95501880296972
45 52.08603112423522
90 84.42178409114511
90 87.4703663990154
60 66.66448537074183
60 62.207214562675354
75 73.13524307962298
45 55.23177441544557
75 78.65299849082874
90 89.95863974509228
60 62.111871344380795
60 73.16282946068486
60 71.10490486153532
75 76.48982982145003
90 78.07403724785866
75 87.35105452116488
75 83.59675182221645
60 55.21050115438212
90 95.31575827521391
45 43.006870524749004
90 91.33365176658529
75 71.3610680253856
90 101.0277256056414
75 92.08734226947259
75 84.48528132038874
90 101.23327741640912
45 45.41114054407731
45 44.60428075177606
60 73.56904619435096
75 74.99403194913997
60 73.79144439988457
45 42.18947745768007
45 52.28524554419357
75 81.33096677835184
75 74.72630939629744
90 74.16493595429628
75 73.9302947672208
60 52.06431678126992
45 48.44066332103973
75 78.65299849082874
75 73.13524307962298
90 93.431556

In [24]:
# AGORA COM A ELEVACAO
regressorElevacao = LinearRegression().fit(xTrain, yTrainElevacao)
yPredElevacao = regressorAzimutal.predict(xTest)

for i, predicao in enumerate(yPredElevacao):
    print(yTestElevacao[i], predicao)

-30 55.63241747408111
-15 88.49189769645386
-30 56.58551317587524
-15 82.86782914247752
0 68.36803847514072
-30 39.07923320253238
0 72.95501880296972
-15 52.08603112423522
0 84.42178409114511
-15 87.4703663990154
-15 66.66448537074183
-15 62.207214562675354
-30 73.13524307962298
-15 55.23177441544557
-15 78.65299849082874
0 89.95863974509228
-30 62.111871344380795
-15 73.16282946068486
0 71.10490486153532
0 76.48982982145003
-15 78.07403724785866
0 87.35105452116488
-15 83.59675182221645
-30 55.21050115438212
-15 95.31575827521391
-15 43.006870524749004
0 91.33365176658529
0 71.3610680253856
0 101.0277256056414
0 92.08734226947259
-15 84.48528132038874
-15 101.23327741640912
-15 45.41114054407731
0 44.60428075177606
-15 73.56904619435096
-15 74.99403194913997
0 73.79144439988457
-15 42.18947745768007
0 52.28524554419357
-30 81.33096677835184
0 74.72630939629744
-30 74.16493595429628
-30 73.9302947672208
-30 52.06431678126992
-30 48.44066332103973
-15 78.65299849082874
-30 73.1352430796

In [25]:
yTestElevacao

array([-30, -15, -30, -15,   0, -30,   0, -15,   0, -15, -15, -15, -30,
       -15, -15,   0, -30, -15,   0,   0, -15,   0, -15, -30, -15, -15,
         0,   0,   0,   0, -15, -15, -15,   0, -15, -15,   0, -15,   0,
       -30,   0, -30, -30, -30, -30, -15, -30, -15,   0,   0, -15, -30,
       -30, -15,   0, -30, -15, -15, -30,   0, -30, -15,   0, -30, -30,
         0, -15, -30,   0,   0,   0,   0,   0,   0, -30, -15,   0,   0,
       -15,   0,   0, -30,   0,   0,   0,   0, -15,   0, -30, -15,   0,
         0, -15,   0, -15, -30, -15,   0,   0, -30, -30,   0, -30, -15,
       -15, -30, -15, -30,   0, -30, -30, -30, -30, -15,   0,   0, -30,
         0, -15,   0, -15, -15, -30, -30, -30, -30, -30,   0, -30, -30,
       -30, -15, -15, -30, -15, -30, -30, -15, -15,   0,   0, -30,   0,
       -15, -30,   0,   0, -15,   0, -15, -30, -15, -15, -30, -15, -15,
       -30,   0,   0,   0, -30, -30, -15, -15,   0, -15, -30, -30, -30,
         0, -30, -15, -30,   0, -30, -15,   0, -15, -30, -30, -1

In [26]:
yTrainElevacao

array([-90, -80, -70, -60, -50, -40, -30, -20, -10,   0, -90, -80, -70,
       -60, -50, -40, -30, -20, -10,   0, -90, -80, -70, -60, -50, -40,
       -30, -20, -10,   0, -90, -80, -70, -60, -50, -40, -30, -20, -10,
         0, -90, -80, -70, -60, -50, -40, -30, -20, -10,   0, -90, -80,
       -70, -60, -50, -40, -30, -20, -10,   0, -90, -80, -70, -60, -50,
       -40, -30, -20, -10,   0, -90, -80, -70, -60, -50, -40, -30, -20,
       -10,   0, -90, -80, -70, -60, -50, -40, -30, -20, -10,   0, -90,
       -80, -70, -60, -50, -40, -30, -20, -10,   0])